Imports necessary libraries

In [1]:
from google.colab import drive
from tqdm import tqdm
import networkx as nx
import nltk
import pickle
import sys

In [2]:
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Mounts Drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


Creates threeStageAlgorithm function and getDeltaQ helper function

In [4]:
def getDeltaQ(key1, key2, communities, node2degrees, graph, m):
  deltaQ = 0
  for vi in communities[key1]:
    neighbors = set(graph.adj[vi])
    for vj in communities[key2]:
      aij = 1 if vj in neighbors else 0
      deltaQ += (aij - node2degrees[vi]*node2degrees[vj]/(2*m))
  return deltaQ / (2*m)

# Takes in an undirected graph, and returns a dictionary where each key is a
# central node of a community and each value is a list of nodes in that
# community
def threeStageAlgorithm(graph, saveDistances=False, pathToDistances=None, saveSimilarities=False, pathToSimilarities=None):
  # Gets the number of nodes along with the degree of each node
  minNode = min(graph.nodes)
  maxNode = max(graph.nodes)
  degrees = [(k,v) for k,v in graph.degree]
  degrees.sort(key=lambda x : x[1],reverse=True)
  node2degrees = {k:v for k,v in degrees}
  n = len(degrees)

  # Creates an array of all distances between pairs of nodes if necessary
  if pathToDistances == None:
    distances = [[None for _ in range(n+1)] for _ in range(n+1)]
    for u in tqdm(range(minNode, maxNode), leave=False, desc='Creating distances array'):
      for v in range(u+1,maxNode+1):
        try:
          pathLength = len(nx.shortest_path(graph, u, v))-1
          distances[u][v] = pathLength
          distances[v][u] = pathLength
        except:
          continue
    
    if saveDistances:
      with open(pathToDistances, 'wb') as f:
        pickle.dump(distances, f)
  
  # Read in an array of the distances between each pair of nodes
  else:
    with open(pathToDistances, 'rb') as f:
      distances = pickle.load(f)

  # Finds the number of pairs that are not connected
  notConnected = 0
  for u in range(minNode,minNode):
    for v in range(u+1,maxNode+1):
      if distances[u][v] == None:
        notConnected += 1
  
  connected = n*(n-1) - notConnected

  # Finds the sum of the distances
  sumDists = 0
  for u in range(minNode,maxNode):
    for v in range(u+1,maxNode+1):
      if distances[u][v] != None:
        sumDists += distances[u][v]

  # Finds the average distance between every pair of connected nodes
  D = 2*sumDists/connected

  # Selected all the central nodes and places them in set C0
  C0 = set([degrees[0][0]])
  for vj, _ in tqdm(degrees[1:], leave=False, desc='Finding central nodes'):
    for v in C0:
      if distances[v][vj] == None or distances[v][vj] >= D:
        C0.add(vj)
        break
  
  # Creates the dictionary of communities
  communities = {n:[n] for n in C0}

  # Creates an array of similarities between every pair of nodes in a graph if
  # necessary
  if pathToSimilarities == None:
    similarities = [[None for _ in range(n+1)] for _ in range(n+1)]
    for u in tqdm(range(minNode,maxNode), leave=False, desc='Creating similarities array'):
      uNeighbors = graph.adj[u]
      uDegree = node2degrees[u]
      for v in range(u+1,maxNode+1):
        vNeighbors = graph.adj[v]
        vDegree = node2degrees[v]
        if (uDegree + vDegree) > 0:
          similarities[u][v] = 2*len(list(set(uNeighbors).intersection(set(vNeighbors))))/(uDegree + vDegree)
          similarities[v][u] = similarities[u][v]
    if saveSimilarities:
      with open(pathToSimilarities, 'wb') as f:
        pickle.dump(similarities, f)

  # Reads in an array of similarities between every pair of nodes in a graph
  else:
    with open(pathToSimilarities,'rb') as f:
      similarities = pickle.load(f)

  # Adds each node that has not been labeled yet to a pre-community
  notLabeled = set(list(range(minNode,maxNode+1))) - C0
  for _ in tqdm(range(len(list(notLabeled))), leave=False, desc='Adding nodes to communities'):
    maxSim = 0
    for v in C0:
      for vj in notLabeled:
        if similarities[v][vj] != None and similarities[v][vj] > maxSim:
          bestCentralNode = v
          bestNewNode = vj
          maxSim = similarities[v][vj]
    communities[bestCentralNode].append(vj)
    notLabeled.remove(vj)

  # Deletes distances and similarities arrays because they will no longer be
  # used (and they're usually very large)
  del distances
  del similarities

  # Calculates the average degree of every node divided by 2
  m = 0
  for v in range(minNode,maxNode+1):
    m += node2degrees[v]
  m /= 2

  # Calculates the change in modularity we would get if we merged each pair of
  # communities
  deltaQs = []
  keys = list(communities.keys())
  merged = {k:False for k in keys}
  for i in tqdm(range(len(keys)-1),leave=False, desc='Calculating changes in modularities'):
    key1 = keys[i]
    for j in range(i+1, len(keys)):
      key2 = keys[j]
      deltaQ = getDeltaQ(key1, key2, communities, node2degrees, graph, m)
      if deltaQ >= 0:
        deltaQs.append([deltaQ, key1, key2])

  # Merges communities to maximize modularity
  deltaQs.sort(key=lambda x : x[0], reverse=True)
  length = len(deltaQs)
  idx = 0
  for idx in tqdm(range(length), leave=False, desc='Merging communities'):
    key1 = deltaQs[idx][1]
    key2 = deltaQs[idx][2]
    if not(merged[key1]) and not(merged[key2]):
      merged[key1] = True
      merged[key2] = True
      communities[key1] = communities[key1] + communities[key2]
      communities.pop(key2)
  
  new_communities = {}
  for idx, item in enumerate(communities.items()):
    new_communities[idx] = item[1]

  return new_communities

Reads docmap file and generates a list of all documents. Each document is represented as a list with 3 elements: the node index, the document id, and the document name

In [5]:
docMapPath = 'drive/My Drive/CS 6850 Final Project/Data/docmap'
file=open(docMapPath,"r")
docMapStrings = file.read().split('\n')
docMapStrings = list(map(lambda x : x.split('\t'), docMapStrings))
docMap = []
for index, line in enumerate(docMapStrings):
  docMap.append([index+1, int(line[0]), line[1]])

Reads pair_doc file and generate a list of all the edges. Each edge is represented as a list of 2 elements: the source node and the destination node.

In [6]:
edgesPath = 'drive/My Drive/CS 6850 Final Project/Data/pair_doc'
file=open(edgesPath,"r")
edgesStrings = file.read().split('\n')
edgesStrings = list(map(lambda x : x.split(' '), edgesStrings))
edgesStrings=  list(filter(lambda x : x != [''], edgesStrings))
edges = []
for line in edgesStrings:
  edges.append([int(line[0]), int(line[1])])

Checks if the node indices are 0-indexed or 1-indexed by seeing if 0 is used in the edges.

In [7]:
for edge in edges:
  if 0 in edge:
    print(edge)

Creates a directed graph, directedG, and an undirected graph, undirectedG1. directedG will contains all edges in the original dataset. undirectedG1 will only contain edges between nodes u and v if there is one edge going from u to v and one edge going from v to u in the dataset.

In [8]:
directedG = nx.DiGraph()
undirectedG1 = nx.Graph()

Adds all nodes to both graphs

In [9]:
for doc in docMap:
  directedG.add_node(doc[0], id=doc[1], name=doc[2])
  undirectedG1.add_node(doc[0], id=doc[1], name=doc[2])

Adds all edges to directedG, and adds necessary edges to undirectedG1

In [10]:
edgesSeen = set()
for edge in edges:
  u = edge[0]
  v = edge[1]
  directedG.add_edge(u, v)
  if str([v, u]) in edgesSeen:
    undirectedG1.add_edge(u, v)
  else:
    edgesSeen.add(str(edge))

Creates another undirected graph, undirectedG2. This graph contains all the edges from directedG, except that they have been converted to undirected edges, and duplicate undirected edges between nodes were removed

In [11]:
undirectedG2 = directedG.to_undirected()

In [12]:
print(directedG.number_of_edges(), undirectedG1.number_of_edges() + undirectedG2.number_of_edges())

22432 22432


Makes sure each graph has the correct number of edges

In [13]:
edgesSeen = set()
undirectedG1Edges = undirectedG1.edges
undirectedG2Edges = undirectedG2.edges
for edge in directedG.edges:
  u = edge[0]
  v = edge[1]
  assert (edge in undirectedG2Edges)
  if str((v,u)) in edgesSeen:
    assert (edge in undirectedG1Edges)
  else:
    edgesSeen.add(str(edge))

In [14]:
notConnected = 0
for node in undirectedG2.nodes:
  if set() == set(undirectedG2.adj[node]):
    notConnected+= 1
print(notConnected)

5107


In [15]:
layers = [undirectedG2]
curCommunities = threeStageAlgorithm(undirectedG2, pathToDistances='drive/My Drive/CS 6850 Final Project/Data/distances.pkl', pathToSimilarities='drive/My Drive/CS 6850 Final Project/Data/similarities.pkl')
communities = [curCommunities]

In [16]:
def generateLayerGraph(communities, childGraph):
  graph = nx.Graph()
  keys = list(communities.keys())
  for k in keys:
    graph.add_node(k)
  
  length = len(keys)
  for u in range(length-1):
    key1 = keys[u]
    uNodes = set(communities[key1])
    for v in range(u+1,length):
      key2 = keys[v]
      for vNode in communities[key2]:
        vNeighbors = set(childGraph.adj[vNode])
        if uNodes.intersection(vNeighbors) != set():
          graph.add_edge(u,v)
          break
  
  return graph

In [17]:
layers.append(generateLayerGraph(communities[0], undirectedG2))

In [18]:
prevLayerLength = len(list(layers[0].nodes))
iteration = 0
while len(communities[-1].items()) > 1 and len(communities[-1].items()) < prevLayerLength:
  iteration += 1
  print(iteration)
  prevLayerLength = len(list(layers[-1].nodes))
  communities.append(threeStageAlgorithm(layers[-1]))
  layers.append(generateLayerGraph(communities[-1], layers[-1]))

1


2


3


4


5


In [19]:
depth = len(layers) - 1

In [ ]:
totalNodes = 0
for graph in layers:
  print(len(list(graph.nodes)))
  totalNodes += len(list(graph.nodes))
print(totalNodes)

Creates a function that takes in a list of layers and communitites an creates a tree

In [21]:
def combineLayers(layers, communities):
  graph = nx.DiGraph()
  n = len(layers)
  root = str((n, 0))
  graph.add_node(root)
  layers.reverse()
  communities.reverse()
  treeLayer = n
  for idx, curGraph in enumerate(layers[1:-1]):
    treeLayer = treeLayer - 1
    for node in curGraph.nodes:
      graph.add_node(str((treeLayer, node)))

    curCommunities= communities[idx]
    for k, v in curCommunities.items():
      for node in v:
        graph.add_edge(str((treeLayer+1, k)), str((treeLayer, node)))
    
  for node in layers[-1].nodes:
    graph.add_node(node)
  
  curCommunities = communities[-1]
  for k, v in curCommunities.items():
    for node in v:
      graph.add_edge(str((2, k)), node)

  return graph, root

In [22]:
tree, root = combineLayers(layers, communities)

In [ ]:
with open('drive/My Drive/CS 6850 Final Project/Data/TSAtree.pkl', 'wb') as f:
  pickle.dump(tree, f)
with open('drive/My Drive/CS 6850 Final Project/Data/TSARoot.pkl', 'wb') as f:
  pickle.dump(root, f)

In [23]:
totalBranches = 0
for node in tree.nodes:
  totalBranches += len(list(tree.successors(node)))
averageBranchingFactor = totalBranches/(totalNodes-len(list(undirectedG2.nodes)))
print(averageBranchingFactor)

1.8404694126532273


In [25]:
names = list(map(lambda x : x[2].replace('_', ' '), docMap))
print(names[:5])

['Computer science', 'Topic outline of computer science', 'Computer scientist', 'Program (mathematical object)', 'Overlapping subproblem']


In [28]:
totalWords = 0
words = []
for name in names:
  word = wn.synsets(name)
  if len(word) == 1:
    print(name)

Informatics
Algorithm
Database
Password
Recursion
Serialization
Identifier
TRIPOS
Deadlock
COMAL
Gambas
Usability
Locale
Flowchart
EECS
Scrolling
ROAM
Workspace
KAON
PRIMOS
Quiesce
Newsvendor
Pintos
CapROS
CVSS
GeckOS
MoL
Snarfing
Thumbnail
Byte
Biostatistics
Cybernetics
Client-server
COBOL
Cyborg
Concept
Cartography
Complexity
Cyberspace
Checksum
Decipherment
Ergonomics
Emoticon
Encryption
Espionage
Fortran
FIFO
GNU
Hexadecimal
Intension
Kludge
Kilobit
MUMPS
Microcode
MS-DOS
Minicomputer
Octal
Operand
Oxymoron
Prolog
Pixel
Programmer
Proteome
Retronym
Robot
Spyware
Thesaurus
Trackball
Unix
WYSIWYG
Supercomputer
Intranet
Pseudonym
Subroutine
Decrypt
Handshaking
Provisioning
Robotics
Wavelet
Proteomics
Mebibyte
Workstation
Metonymy
KRYPTON
Webcam
Gibibyte
Tebibyte
Exbibyte
Kibibyte
Homophone
TWAIN
Polysemy
Laptop
Pleonasm
TADS
Backtracking
Nomogram
Hemodynamics
Allusion
Telerobotics
Humanoid
Parsing
Trigram
Debugging
Megabit
Pebibyte
Gigabit
Mousepad
Self-destruct
HighLife
PaX
Dependabi

In [27]:
print(words)

[Synset('information_science.n.01'), Synset('algorithm.n.01'), Synset('database.n.01'), Synset('password.n.01'), Synset('recursion.n.01'), Synset('serialization.n.01'), Synset('identifier.n.01'), Synset('tripos.n.01'), Synset('deadlock.n.01'), Synset('comate.s.02'), Synset('viola_da_gamba.n.01'), Synset('serviceability.n.01'), Synset('venue.n.01'), Synset('flow_chart.n.01'), Synset('european_union.n.01'), Synset('scroll.v.01'), Synset('roll.v.12'), Synset('workspace.n.01'), Synset('kaon.n.01'), Synset('primo.n.01'), Synset('quieten.v.01'), Synset('newsagent.n.01'), Synset('pinto.n.01'), Synset('capros.n.01'), Synset('curriculum_vitae.n.01'), Synset('gecko.n.01'), Synset('gram_molecule.n.01'), Synset('pilfer.v.01'), Synset('thumbnail.n.01'), Synset('byte.n.01'), Synset('biometrics.n.01'), Synset('cybernetics.n.01'), Synset('client-server.a.01'), Synset('cobol.n.01'), Synset('cyborg.n.01'), Synset('concept.n.01'), Synset('mapmaking.n.01'), Synset('complexity.n.01'), Synset('internet.n.01

In [ ]:
n = len(words)
wordnetSimilarities = []
for i in range(n-1):
  print(words[i])
  input()
  for j in range(i+1,n):
    print(words[i].wup_similarity(words[j]))

Information about three-stage algorithm tree:

Depth: 5

Average branching factor: 1.84
